In [ ]:
# Compliance is defined as the spectral ratio between pressure and vertical displacement data.
# Compliance noise arises from seafloor deformation due to seafloor and water wave effects (including infragravity waves).
# This is likely the main source of noise in vertical component OBS data.
# This analysis therefore requires both vertical (?HZ) and pressure (?XH) data.

# Tilt noise arises from OBS stations that are not perfectly leveled, and therefore the horizontal seafloor deformation leaks onto the vertical component.
# This effect can be removed by calculating the spectral ratio between horizontal and vertical displacement data.
# In most cases, however, the tilt direction (measured on a compass - as opposed to tilt angle, measured from the vertical axis) is unknown
# and must be determined from the coherence between rotated horizontal components and the vertical component.

# ~/opt/anaconda3/envs/Seismic_Noise_Data_Obspy_ML_NN/lib/python3.8/site-packages/obstools

# CI-7D Stations Meta
# http://service.iris.edu/irisws/fedcatalog/1/query?net=7D&starttime=2011-01-01&endtime=2017-12-31&format=text&includeoverlaps=true&nodata=404


In [ ]:
from obstools.atacr import DayNoise, TFNoise, EventStream, StaNoise, utils
import obstools.atacr.plotting as atplot
from pathlib import Path
from obspy.core import read, Stream, Trace, AttribDict, UTCDateTime

### ATaCR_Python Step-by-Step Process:
The entire ATaCR code can be implimented to clean event data with the following seven scripts, in order:

                                        1. query_fdsn_stdb

                                        2. atacr_download_event

                                        3. atacr_download_data

                                        4. atacr_daily_spectra

                                        5. atacr_clean_spectra

                                        6. atacr_transfer_functions
                                        
                                        7. atacr_correct_event

[1.] <samp><ins><< Define Station Metadata >></ins></samp> 

Query and save a list of stations to an stdb object in a pickle file:

```
!query_fdsn_stdb -N '{N}' -C '{C}' -S {' '.join(S)} ./Data/sta_query> ./Data/Step_1_7_StationMeta_logfile.log
```
You can confirm the station list with:

```
!ls_stdb ./Data/sta_query.pkl
```

[2.] <samp><ins><< Download Event Data >></ins></samp> 

With a general idea on the station list monitoring window, query a list of events:

```
!atacr_download_event --min-mag={Minmag} --max-mag={Maxmag} --start='{EventStart}' --end='{EventEnd}' ./Data/sta_query.pkl> ./Data/Step_2_7_EventDownload_logfile.log
```

[3.] <samp><ins><< Download Noise Data >></ins></samp> 

Download the time window for the generally non-transient passive data over the entire station list:
```
!atacr_download_data --start='{NoiseStart}' --end='{NoiseEnd}' ./Data/sta_query.pkl> ./Data/Step_3_7_NoiseDownload_logfile.log
```

[4.] <samp><ins><< Quality Control Noise Data >></ins></samp> 

Data quality control followed by calculating daily spectral averages for the entire array:
```
!atacr_daily_spectra -O --save-fig --figQC --figAverage --figCoh --start='{SpecStart}' --end='{SpecEnd}' ./Data/sta_query.pkl> ./Data/Step_4_7_QCSpectra_logfile.log
```

[5.] <samp><ins><< Calculate Noise Averages >></ins></samp> 

With daily averages for each station complete, produce a better and the cleanest and most continuous spectral average (ie a month long average) for each station:
```
!atacr_clean_spectra -O --save-fig --figQC --figAverage --figCoh --figCross --start='{SpecStart}' --end='{SpecEnd}' ./Data/sta_query.pkl> ./Data/Step_5_7_CleanSpectra_logfile.log
```

[6.] <samp><ins><< Calculate Transfer Functions >></ins></samp> 

With the cleanest and most complete representation of noise spectra in hand, we can now calculate transfer functions that can remove the noise contaminated between each component pair (ie, ZP-H is the tf that would remove noise from the vertical (ZP) produced by the pressure component (H)):
```
!atacr_transfer_functions ./Data/sta_query.pkl> ./Data/Step_6_7_CalcTFs_logfile.log
```
#
#
```java
        Using the component noise data, six unique transfer functions will be calculated:

                Respective components:

                    Z: Vertical

                    H1: Horizontal channel 1 (e.g. maybe E-N?)

                    H2: Horizontal channel 2 (e.g. maybe N-S?)

                    H: All horizontal noise rotated into into direction with the greatest the spectral density

                    P: Pressure channel

                The Six Transfer Functions Calculated:

                    1. ZP: TF that removes pressure (P) from the vertical (Z).
                    
                    2. Z1: TF that removes horiz-1 (H1) from the vertical (Z).
                    
                    3. Z2-1: TF that removes H2 from the Z AFTER H1 was has been removed.
                    
                    4. ZP-21: TF that removes P from Z AFTER H1 and H2 were removed.
                    
                    5. ZH: TF that removes the maximum horizontal noise (found from grid-search rotations).
                    
                    6. ZP-H: TF that removes P from Z after H was removed.

                The result is a series of filters, transfer functions, that remove all estimated noise spectra contaminating the VERTICAL component by other channels through tilt-noise, rotational-noise, and site-noise.
```

[7.] <samp><ins><< Correct Event Data with Transfer Functions >></ins></samp> 

Now that we have a sufficient approximation of ambient noise-spectra, whether made by tilt/rotation/site-effects/etc., that yields TFs that can remove said noise we can now test the result out on cleaning actual event (downloaded at the start) signal to see the gain in quality:
```
!atacr_correct_event --figRaw --figClean ./Data/sta_query.pkl> ./Data/Step_7_7_CorrectEvents_logfile.log
```



#### Some notes on the plots

<samp><ins>day_corrected.png:</samp></ins> Corrected event traces using the daily spectral average of ALL data available, irrespective of any pre-event time-windows and transients in the data. These are generally not as good as the 'cleaned' spectra.

<samp><ins>sta_corrected.png:</samp></ins> Corrected event traces using the 'cleaned' station spectral average that windows around (SpecAvgStart<=evtime<=SpecAvgEnd) the event time.

<samp><ins>[STA].[DAY].average:</samp></ins> Station spectral average for a single day.

<samp><ins>[STA].average.png:</samp></ins> Station spectral average without any 'cleaning' out the transient spectra or lame time windows.

<samp><ins>coh_ph.png:</samp></ins> Daily coherence and phase as a function of rotation angle from H1. This figure can show the angle that can yield H, the maximum horizontal noise spectra possible for any rotation angle.

<samp><ins>specgram_H1.H2.Z.P.png:</samp></ins> 4-C Spectrograms.

<samp><ins>raw.png:</samp></ins> Raw event trace.

<samp><ins>av_admittance.png:</samp></ins> Station average admittance spectra.

<samp><ins>av_coherence.png:</samp></ins> Station average coherence spectra.

<samp><ins>av_phase.png:</samp></ins> Station average phase spectra.

<samp><ins>QC.png:</samp></ins> All 'cleaned' spectra that will be used for averaging in the TFs that surround an event time.

<samp><ins>transfer_functions.png:</samp></ins> All 'cleaned' transfer functions and their average.


### Some questions
When looking at the cleaned spectra for a shallow OBS it seems apparent that the pressure component shows the 'notch' between the primary and secondary microsism at ~10% higher frequency than its complimentary components (H1,H2,Z). Why? I understand the notch position is controlled primarily by pressure value (P) but why this spectral hysteresis in the notch between the components?


### The following block does a comprehensive run of the entire ATaCR code

In [ ]:
# =============# =============# =============
# =============# ATACR-PARAMETERS # =============
# =============# =============# =============

N='7D' #network
C='?H?' #channels
S= ['M01A','M02A','M07A','M08A'] #station

Minmag=6.3
Maxmag=6.7

EventStart='2012-03-01 00:00:00' #event start date
EventEnd='2012-03-10 00:00:00' #event end date

NoiseStart='2012-03-01 00:00:00' #noise start date
NoiseEnd='2012-03-10 00:00:00' #noise end date

SpecStart='2012-03-01 00:00:00' #spectral-averaging start date
SpecEnd='2012-03-10 00:00:00' #spectral-averaging end date

# STEPS = [1,2,3,4,5,6,7] #Absolutely every step - Downloading adds an hour or more to the process
STEPS = [1,4,5,6,7] #Everything but the download steps - About 4min for six stations.
ClearProcessedData = True

# =============# =============# =============
# =============# =============# =============
# =============# =============# =============



PyATaCTPath = !pwd
PyATaCTPath = PyATaCTPath[0]
if ClearProcessedData:
    !rm -rf {PyATaCTPath +  '/AVG_STA/'}
    !rm -rf {PyATaCTPath +  '/SPECTRA/'}
    !rm -rf {PyATaCTPath +  '/TF_STA'}
if 1 in STEPS:
    print('Step 1/7 - BEGIN: Station Metadata')
    !query_fdsn_stdb -N '{N}' -C '{C}' -S {','.join(S)} ./Data/sta_query> ./Data/Step_1_7_StationMeta_logfile.log
    # !ls_stdb ./Data/sta_query.pkl
    print('Step 1/7 - COMPLETE: Station Metadata')
if 2 in STEPS:
    print('Step 2/7 - BEGIN: Download Event Data')
    !atacr_download_event --min-mag={Minmag} --max-mag={Maxmag} --start='{EventStart}' --end='{EventEnd}' ./Data/sta_query.pkl> ./Data/Step_2_7_EventDownload_logfile.log
    print('Step 2/7 - COMPLETE: Download Event Data')
if 3 in STEPS:
    print('Step 3/7 - BEGIN: Download Day Data')
    !atacr_download_data --start='{NoiseStart}' --end='{NoiseEnd}' ./Data/sta_query.pkl> ./Data/Step_3_7_NoiseDownload_logfile.log
    print('Step 3/7 - COMPLETE: Download Day Data')
if 4 in STEPS:
    print('Step 4/7 - BEGIN: Quality Control Noise Data')
    !atacr_daily_spectra -O --figQC --figAverage --figCoh --save-fig --start='{SpecStart}' --end='{SpecEnd}' ./Data/sta_query.pkl> ./Data/Step_4_7_QCSpectra_logfile.log
    print('Step 4/7 - COMPLETE: Quality Control Noise Data')
if 5 in STEPS:
    print('Step 5/7 - BEGIN: Spectral Average of Noise Data')
    !atacr_clean_spectra -O --figQC --figAverage --figCoh --figCross --save-fig --start='{SpecStart}' --end='{SpecEnd}' ./Data/sta_query.pkl> ./Data/Step_5_7_CleanSpectra_logfile.log
    print('Step 5/7 - COMPLETE: Spectral Average of Noise Data')
if 6 in STEPS:
    print('Step 6/7 - BEGIN: Calculate Transfer Functions')
    !atacr_transfer_functions -O --figTF --save-fig ./Data/sta_query.pkl> ./Data/Step_6_7_CalcTFs_logfile.log
    print('Step 6/7 - COMPLETE: Calculate Transfer Functions')
if 7 in STEPS:
    print('Step 7/7 - BEGIN: Correct Event Data')
    !atacr_correct_event --figRaw --figClean --save-fig ./Data/sta_query.pkl> ./Data/Step_7_7_CorrectEvents_logfile.log
    print('Step 7/7 - COMPLETE: Correct Event Data')


In [ ]:
# for sta in S:
#     folder = './Data/' + N + '.' + sta
#     trN1, trN2, trNZ, trNP = utils.get_data(Path(folder), UTCDateTime(Start), UTCDateTime(End))
#     daynoise = DayNoise(trN1.merge().split()[0], trN2.merge().split()[0], trNZ.merge().split()[0], trNP.merge().split()[0])
#     daynoise.QC_daily_spectra()
#     # daynoise.QC_daily_spectra(fig_QC=True)
#     daynoise.average_daily_spectra()
#     tfnoise_day = TFNoise(daynoise)
#     tfnoise_day.transfer_func()
#     stanoise = StaNoise(daylist=[daynoise,daynoise])
#     stanoise.QC_sta_spectra()
#     stanoise.average_sta_spectra()
#     tfnoise_sta = TFNoise(stanoise)
#     tfnoise_sta.transfer_func()

In [ ]:
# daynoise = DayNoise(trN1.merge().split()[0], trN2.merge().split()[0], trNZ.merge().split()[0], trNP.merge().split()[0])
# daynoise.QC_daily_spectra()
# daynoise.average_daily_spectra()
# tfnoise_day = TFNoise(daynoise)
# tfnoise_day.transfer_func()
# stanoise = StaNoise(daylist=[daynoise])
# stanoise.QC_sta_spectra()
# stanoise.average_sta_spectra()
# tfnoise_sta = TFNoise(stanoise)
# tfnoise_sta.transfer_func()
# daynoise.average_daily_spectra()
# tfnoise_day = TFNoise(daynoise)
# tfnoise_day.transfer_func()
# evstream.correct_data(tfnoise_day)
# figure = atplot.fig_event_corrected(evstream, tfnoise_day.tf_list, fmin=1./150., fmax=2.)
# figure.show()